In [ ]:
#%% Import
!pip install selenium openpyxl
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
import time
import pandas as pd

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 45.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 492.9/492.9 kB 20.3 MB/s eta 0:00:00


In [ ]:
#%% Khởi tạo WebDriver
driver = webdriver.Chrome()
url = "https://cafef.vn/du-lieu/lich-su-giao-dich-symbol-pnj/trang-1-0-tab-1.chn"
driver.get(url)

try:
    WebDriverWait(driver, 20).until(
        EC.presence_of_element_located((By.ID, "render-table-owner"))
    )
    print("Trang đã tải xong.")
except TimeoutException:
    print("Trang không tải hoặc lỗi timeout.")
    driver.quit()

SessionNotCreatedException: Message: session not created: probably user data directory is already in use, please specify a unique value for --user-data-dir argument, or don't use --user-data-dir
Stacktrace:
#0 0x58e23b446ffa <unknown>
#1 0x58e23af05970 <unknown>
#2 0x58e23af3f42a <unknown>
#3 0x58e23af3b18f <unknown>
#4 0x58e23af8bbd9 <unknown>
#5 0x58e23af8b106 <unknown>
#6 0x58e23af7d063 <unknown>
#7 0x58e23af49328 <unknown>
#8 0x58e23af4a491 <unknown>
#9 0x58e23b40e42b <unknown>
#10 0x58e23b4122ec <unknown>
#11 0x58e23b3f5a22 <unknown>
#12 0x58e23b412e64 <unknown>
#13 0x58e23b3d9bef <unknown>
#14 0x58e23b435558 <unknown>
#15 0x58e23b435736 <unknown>
#16 0x58e23b445e76 <unknown>
#17 0x79062eaefac3 <unknown>


In [ ]:
#%% Nhập khoảng thời gian tìm kiếm
date_input = driver.find_element(By.ID, "date-inp-disclosure")
date_input.clear()
date_input.send_keys("01/01/2014 - 31/12/2024")
date_input.send_keys(Keys.ENTER)

driver.execute_script("document.querySelector('#date-inp-disclosure').blur();")
time.sleep(1)

# Nhấn nút "Xem"
find_button = driver.find_element(By.XPATH, '//*[@id="owner-find"]')
find_button.click()

In [ ]:
#%% Chờ dữ liệu tải sau khi bấm tìm kiếm
try:
    WebDriverWait(driver, 10).until(
        EC.visibility_of_all_elements_located((By.CSS_SELECTOR, "#render-table-owner tr"))
    )
    print("Dữ liệu đã tải xong!")
    time.sleep(3)
except TimeoutException:
    print("Không tìm thấy dữ liệu sau khi chờ AJAX!")
    driver.quit()

In [ ]:
#%% Lưu dữ liệu
all_data = []
current_page = 1
max_pages = 138  # Giới hạn số trang

while current_page <= max_pages:
    print(f"Đang xử lý trang {current_page}...")

    # Chờ để dữ liệu tải hoàn toàn trên trang
    try:
        WebDriverWait(driver, 10).until(
            EC.visibility_of_all_elements_located((By.CSS_SELECTOR, "#render-table-owner tr"))
        )
        print(f"Dữ liệu trang {current_page} đã tải xong.")
        time.sleep(2)
    except TimeoutException:
        print(f"Không tìm thấy dữ liệu trên trang {current_page}.")
        break

    # Cuộn xuống để đảm bảo tất cả dữ liệu được tải
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(2)

    # Lấy danh sách `<tr>`
    rows = driver.find_elements(By.CSS_SELECTOR, "#render-table-owner tr")

    print(f"Tìm thấy {len(rows)} dòng dữ liệu trong bảng.")

    # Nếu không tìm thấy dữ liệu, thoát vòng lặp
    if len(rows) == 0:
        break

    # Lấy dữ liệu từ bảng
    for row in rows:
        cols = row.find_elements(By.TAG_NAME, "td")
        if len(cols) >= 11:
            data = {
                "Ngày": cols[0].text.strip(),
                "Giá mở cửa": cols[8].text.strip(),
                "Giá cao nhất": cols[9].text.strip(),
                "Giá thấp nhất": cols[10].text.strip(),
                "Giá đóng cửa": cols[1].text.strip(),
                "Khối lượng": cols[4].text.strip()
            }
            all_data.append(data)

    print(f" Đã thu thập {len(rows)} dòng từ trang {current_page}")

    try:
        next_button = driver.find_element(By.XPATH, '//*[@id="divStart"]/div/div[3]/div[3]')
        if next_button.is_displayed() and next_button.is_enabled():
            next_button.click()
            time.sleep(2)
            current_page += 1
        else:
            print("Next page button not available or disabled.")
            break
    except Exception as e:
        print(f"Error navigating to next page: {e}")
        break

print(f"Tổng dữ liệu thu thập: {len(all_data)} dòng từ {current_page-1} trang.")

In [ ]:
#%% Đóng trình duyệt
driver.quit()

In [ ]:
#%% Lưu dữ liệu vào Excel
df = pd.DataFrame(all_data)
df = df.iloc[1:]
df.to_excel("PNJ_2014_2024.xlsx", index=False, engine='openpyxl')
print(f" Dữ liệu đã được lưu vào file **PNJ_2014_2024.xlsx**!")

